## In develop method for transfer .bin to .npy 

In [1]:
import struct
from struct import unpack
import os

In [3]:
def unpack_drawing(file_handler, object_name):
	
	key_id, = unpack('Q', file_handler.read(8))
	country_code, = unpack('2s', file_handler.read(2))
	recognized, = unpack('b', file_handler.read(1))
	timestamp, = unpack('I', file_handler.read(4))
	n_strokes, = unpack('H', file_handler.read(2))
 
	image = []
	
	for i in range(n_strokes):
		n_points, = unpack('H', file_handler.read(2))
		fmt = str(n_points) + 'B'
		x = unpack(fmt, file_handler.read(n_points))
		y = unpack(fmt, file_handler.read(n_points))
		image.append((x, y))

	return {
		'key': key_id,
		'word': object_name,
		'country_code': country_code,
		'recognized': recognized,
		'timestamp': timestamp,
		'image': image
	}

In [4]:
def unpack_all_drawings(filename):
  file_path = os.path.basename(filename)
  object_name, ext = os.path.splitext(file_path)
  with open(filename, 'rb') as f:
    for i in range(0,4):
      try:
        yield unpack_drawing(f, object_name)
      except struct.error:
        break

In [5]:
for drawing in unpack_all_drawings('./bin_data/apple.bin'):
  print(drawing)

{'key': 6420579601088512, 'word': 'apple', 'country_code': b'US', 'recognized': 0, 'timestamp': 1489184277, 'image': [((255, 255), (0, 0)), ((255, 255), (0, 0)), ((255, 255), (0, 0)), ((255, 254), (0, 1)), ((131, 124, 114, 69, 37, 10, 0, 0, 5, 16, 31, 50, 68, 86, 101, 115, 126, 135, 137, 135, 122, 106), (50, 39, 39, 59, 89, 127, 172, 194, 215, 233, 244, 249, 249, 241, 225, 203, 174, 143, 114, 88, 65, 45)), ((84, 77, 81, 88, 99, 122, 138, 161, 180), (97, 85, 52, 34, 18, 4, 1, 2, 12))]}
{'key': 4986110117675008, 'word': 'apple', 'country_code': b'RU', 'recognized': 1, 'timestamp': 1488954584, 'image': [((95, 79, 68, 31, 17, 9, 1, 0, 4, 54, 103, 130, 168, 190, 204, 219, 228, 222, 210, 200, 194, 197, 203, 192, 189, 190, 209, 207, 197, 179, 107, 100), (62, 50, 49, 74, 91, 113, 163, 220, 226, 249, 255, 255, 244, 231, 215, 187, 152, 151, 160, 160, 156, 140, 137, 134, 115, 109, 88, 82, 71, 63, 62, 65)), ((100, 100, 104, 110, 115, 115, 108, 107), (58, 17, 1, 0, 12, 22, 44, 64))]}
{'key': 648908